In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle


plt.style.use("ggplot")
%matplotlib inline

In [2]:
train_df = pd.read_csv("./train.csv").fillna("sterby")
test_df = pd.read_csv("./test.csv").fillna("sterby")

In [3]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
X_train = train_df["comment_text"].values
y_train = train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test_df["comment_text"].values

In [5]:
#!pip conda keras

In [14]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam

In [7]:
max_features = 20000  # number of words we want to keep
maxlen = 100  # max length of the comments in the model
batch_size = 64  # batch size for the model
embedding_dims = 20  # dimension of the hidden variable, i.e. the embedding dimension

In [8]:
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(list(X_train) + list(X_test))

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

x_train = tok.texts_to_sequences(X_train)
x_test = tok.texts_to_sequences(X_test)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

159571 train sequences
153164 test sequences
Average train sequence length: 65
Average test sequence length: 57


In [9]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (159571, 100)
x_test shape: (153164, 100)


In [10]:
comment_input = Input((maxlen,))

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
comment_emb = Embedding(max_features, embedding_dims, input_length=maxlen, 
                        embeddings_initializer="uniform")(comment_input)

# we add a GlobalMaxPooling1D, which will extract features from the embeddings
# of all words in the comment
h = GlobalMaxPooling1D()(comment_emb)

# We project onto a six-unit output layer, and squash it with a sigmoid:
output = Dense(6, activation='sigmoid')(h)

model = Model(inputs=comment_input, outputs=output)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
143613/143613 [==============================] - 13s 94us/sample - loss: 0.1498 - acc: 0.9601 - val_loss: 0.0740 - val_acc: 0.9761
Epoch 2/5
143613/143613 [==============================] - 12s 80us/sample - loss: 0.0626 - acc: 0.9788 - val_loss: 0.0621 - val_acc: 0.9788
Epoch 3/5
143613/143613 [==============================] - 12s 80us/sample - loss: 0.0546 - acc: 0.9806 - val_loss: 0.0586 - val_acc: 0.9794
Epoch 4/5
143613/143613 [==============================] - 11s 79us/sample - loss: 0.0508 - acc: 0.9816 - val_loss: 0.0570 - val_acc: 0.9797
Epoch 5/5
143613/143613 [==============================] - 12s 81us/sample - loss: 0.0484 - acc: 0.9822 - val_loss: 0.0561 - val_acc: 0.9801


In [12]:
model.save('Simple_toxic_detection.h5')

In [43]:
model = load_model('Simple_toxic_detection.h5')

TypeError: predict() missing 1 required positional argument: 'x'

# теперь та часть что будет использоваться

In [15]:
model = load_model('Simple_toxic_detection.h5')

In [16]:
from io import StringIO
import pandas as pd

TESTDATA = StringIO("you ugly ass nigga")

my_string = pd.read_csv(TESTDATA)

In [21]:

my_string = tok.texts_to_sequences(my_string)
my_string = sequence.pad_sequences(my_string, maxlen=maxlen)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [18]:
model.predict(my_string)

array([[0.99342626, 0.25371426, 0.9204646 , 0.1248413 , 0.88184226,
        0.12688594]], dtype=float32)